In [ ]:
!pip install split-folders

In [ ]:
cd 'drive/My Drive/Brain'

[Errno 2] No such file or directory: 'drive/My Drive/Brain'
/content/drive/My Drive/Brain


In [ ]:
!unzip Img_Dataset_series2.zip

In [ ]:
import splitfolders

splitfolders.ratio("data", output="data_split", seed=1337, ratio=(.8, .2), group_prefix=None)

In [ ]:
import os
from tensorflow.keras.applications import Xception, ResNet50, NASNetLarge, InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD, Nadam

In [ ]:
data_train = "./data_split/train"
data_val = "./data_split/val"
model_path = "./model"
top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
checkpoint = os.path.join(os.path.abspath(model_path), 'checkpoint.h5')

n_classes = 75
# based_model_last_block_layer_number = 403
based_model_last_block_layer_number = 36
# img_width, img_height = 331, 331
img_width, img_height = 299, 299
# img_width, img_height = 224, 224
batch_size = 128
n_epoch = 50
lr = 1e-5
momentum = 0.9

In [ ]:
base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
# x = Dense(2048, activation='relu')(x)
# x = Dropout(0.5)(x)
predictions = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for i, layer in enumerate(base_model.layers):
    layer.trainable = False
    print(i, layer.name)

# print(model.summary())

0 input_2
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_4
13 block2_pool
14 batch_normalization_4
15 add_12
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_5
23 block3_pool
24 batch_normalization_5
25 add_13
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_6
33 block4_pool
34 batch_normalization_6
35 add_14
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_15
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6_se

In [ ]:
base_model = InceptionResNetV2(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
# x = Dense(2048, activation='relu')(x)
# x = Dropout(0.5)(x)
predictions = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for i, layer in enumerate(base_model.layers):
    layer.trainable = False
    print(i, layer.name)

#print(model.summary())

In [ ]:
base_model = NASNetLarge(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
# x = Dense(2048, activation='relu')(x)
# x = Dropout(0.5)(x)
predictions = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for i, layer in enumerate(base_model.layers):
    layer.trainable = False
    # print(i, layer.name)

# print(model.summary())

In [ ]:
base_model = ResNet50(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(n_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for i, layer in enumerate(base_model.layers):
    layer.trainable = False
    print(i, layer.name)

# print(model.summary())

In [ ]:
train_datagen = image.ImageDataGenerator(rescale=1./255,
                             rotation_range=5,
                             shear_range=0.1,
                             zoom_range=0.1,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             brightness_range=[0.7,1.3],
                             horizontal_flip=True)

val_datagen = image.ImageDataGenerator(rescale=1./255)

train_flow = train_datagen.flow_from_directory(
    data_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

val_flow = val_datagen.flow_from_directory(
    data_val,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

n_train = train_flow.n
n_val = val_flow.n

Found 2465 images belonging to 75 classes.
Found 640 images belonging to 75 classes.


In [ ]:
model.compile(optimizer='nadam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_accuracy', patience=5, verbose=0)
]

model.fit_generator(train_flow,
                    steps_per_epoch=n_train//batch_size,
                    epochs=int(n_epoch),
                    validation_data=val_flow,
                    validation_steps=n_val//batch_size,
                    callbacks=callbacks_list)

Epoch 1/50
19/19 [==============================] - ETA: 0s - loss: 4.3129 - accuracy: 0.0141
Epoch 00001: val_accuracy improved from -inf to 0.02344, saving model to /content/drive/My Drive/Brain/model/top_model_weights_InceptionResNetV2.h5
19/19 [==============================] - 157s 8s/step - loss: 4.3129 - accuracy: 0.0141 - val_loss: 4.2994 - val_accuracy: 0.0234
Epoch 2/50
19/19 [==============================] - ETA: 0s - loss: 4.2911 - accuracy: 0.0163
Epoch 00002: val_accuracy did not improve from 0.02344
19/19 [==============================] - 151s 8s/step - loss: 4.2911 - accuracy: 0.0163 - val_loss: 4.2798 - val_accuracy: 0.0156
Epoch 3/50
19/19 [==============================] - ETA: 0s - loss: 4.2821 - accuracy: 0.0154
Epoch 00003: val_accuracy did not improve from 0.02344
19/19 [==============================] - 146s 8s/step - loss: 4.2821 - accuracy: 0.0154 - val_loss: 4.2734 - val_accuracy: 0.0156
Epoch 4/50
19/19 [==============================] - ETA: 0s - loss: 4.

In [ ]:
# model.load_weights(top_weights_path)
model.load_weights(checkpoint)

for layer in model.layers[:based_model_last_block_layer_number]:
    layer.trainable = False
for layer in model.layers[based_model_last_block_layer_number:]:
    layer.trainable = True

# opt = SGD(learning_rate=lr, momentum=momentum)
opt = Nadam(learning_rate=lr)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=5, verbose=0)
]

model.fit_generator(train_flow,
                    steps_per_epoch=n_train//batch_size,
                    epochs=n_epoch,
                    validation_data=val_flow,
                    validation_steps=n_val//batch_size,
                    callbacks=callbacks_list)

model_json = model.to_json()
with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
    json_file.write(model_json)

Epoch 1/50
19/19 [==============================] - ETA: 0s - loss: 0.5766 - accuracy: 0.8622
Epoch 00001: val_accuracy improved from -inf to 0.76250, saving model to /content/drive/My Drive/Brain/model/model_weights.h5
19/19 [==============================] - 171s 9s/step - loss: 0.5766 - accuracy: 0.8622 - val_loss: 0.8704 - val_accuracy: 0.7625
Epoch 2/50
19/19 [==============================] - ETA: 0s - loss: 0.5437 - accuracy: 0.8686
Epoch 00002: val_accuracy improved from 0.76250 to 0.76406, saving model to /content/drive/My Drive/Brain/model/model_weights.h5
19/19 [==============================] - 159s 8s/step - loss: 0.5437 - accuracy: 0.8686 - val_loss: 0.8547 - val_accuracy: 0.7641
Epoch 3/50
19/19 [==============================] - ETA: 0s - loss: 0.5440 - accuracy: 0.8695
Epoch 00003: val_accuracy did not improve from 0.76406
19/19 [==============================] - 154s 8s/step - loss: 0.5440 - accuracy: 0.8695 - val_loss: 0.8452 - val_accuracy: 0.7594
Epoch 4/50
19/19 [